# Building AI Agents with OCI and Llama Stack

This notebook demonstrates how to build Agents using Oracle Cloud Infrastructure (OCI) Generative AI models through Llama Stack.

## What You'll Learn

1. **Basic Agents** - Create agents with custom instructions
2. **Multi-Turn Conversations** - Agents that maintain context across interactions
3. **Agentic Workflows** - Chain multiple agent capabilities together

## Prerequisites

1. **Configure OCI credentials:**
   - Set up `~/.oci/config` with your OCI credentials
   - Set the `OCI_COMPARTMENT_OCID` environment variable
   - Set the `OCI_REGION` environment variable
   - Set the `OCI_AUTH_TYPE` environment variable with 'config_file'

2. **Start Llama Stack server with OCI provider:**
   ```bash
   llama stack run /path/to/your_oci_config.yaml
   ```

## Setup and Configuration

First, install the required packages:

In [19]:
# Install required packages
!pip install llama-stack-client oci

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [20]:

# Optional: Configure custom venv path if needed
import sys
import os

venv_path = os.getenv('LLAMA_STACK_VENV_PATH')
if venv_path:
    sys.path.insert(0, venv_path)
    print(f"✅ Python path updated to use venv: {venv_path}")

# Verify llama-stack-client is installed
try:
    import llama_stack_client
    print(f"✅ llama-stack-client version: {llama_stack_client.__version__}")
except ImportError:
    print("⚠️  llama-stack-client not found. Please install with: pip install llama-stack-client")
    raise

✅ llama-stack-client version: 0.4.0-alpha.1


In [37]:
# Import required libraries
from llama_stack_client import LlamaStackClient, Agent
import os
import json

# Check required OCI environment variables
os.environ["OCI_COMPARTMENT_OCID"] = "ocid1.tenancy.oc1..xxx" # set it with your compartment id
os.environ["OCI_REGION"] = "us-phoenix-1" # change it to your default region
os.environ["OCI_AUTH_TYPE"] = "config_file"
required_env_vars = {
    "OCI_COMPARTMENT_OCID": "export OCI_COMPARTMENT_OCID='ocid1.compartment.oc1..xxx'",
    "OCI_REGION": "export OCI_REGION='us-phoenix-1'",
    "OCI_AUTH_TYPE": "export OCI_AUTH_TYPE='config_file'",
}

missing_vars = []
for var, example in required_env_vars.items():
    if not os.getenv(var):
        missing_vars.append(f"  • {var}: {example}")
    else:
        print(f"✅ {var} is set")

if missing_vars:
    print("\n⚠️  WARNING: Missing required environment variables:")
    for msg in missing_vars:
        print(msg)

# Initialize client
client = LlamaStackClient(base_url="http://localhost:8321")
print("\n✅ Connected to Llama Stack server")

# Get available models
models = client.models.list()
if not models:
    raise RuntimeError("No models available. Please check your Llama Stack server configuration.")

print(f"\n📋 Available models:")
for model in models:
    print(f"   • {model.id}")

# Select primary model
model_id = models[0].id
print(f"\n🎯 Using model: {model_id}")

INFO:httpx:HTTP Request: GET http://localhost:8321/v1/models "HTTP/1.1 200 OK"


✅ OCI_COMPARTMENT_OCID is set
✅ OCI_REGION is set
✅ OCI_AUTH_TYPE is set

✅ Connected to Llama Stack server

📋 Available models:
   • oci/google.gemini-2.5-flash
   • oci/google.gemini-2.5-pro
   • oci/google.gemini-2.5-flash-lite
   • oci/xai.grok-4-fast-non-reasoning
   • oci/xai.grok-4-fast-reasoning
   • oci/xai.grok-code-fast-1
   • oci/xai.grok-4
   • oci/xai.grok-3-mini-fast
   • oci/xai.grok-3-fast
   • oci/xai.grok-3
   • oci/xai.grok-3-mini

🎯 Using model: oci/google.gemini-2.5-flash


---
## 1. Creating a Basic Agent

Agents in Llama Stack are autonomous AI systems with:
- **Instructions**: System-level guidance that shapes behavior
- **Sessions**: Conversation contexts that maintain history
- **Turns**: Individual interactions within a session

In [22]:
# Session tracking and cleanup utilities
all_sessions: list[str] = []

def track_session(session_id: str) -> str:
    """Track a session for later cleanup."""
    all_sessions.append(session_id)
    return session_id

def cleanup_sessions(session_ids: list[str]) -> None:
    """
    Clean up sessions after use.

    Args:
        session_ids: List of session IDs to delete
    """
    if not session_ids:
        print("No sessions to clean up.")
        return

    print(f"🧹 Cleaning up {len(session_ids)} session(s)...")
    for sid in session_ids:
        try:
            client.conversations.delete(conversation_id=sid)
            print(f"   ✅ Deleted session: {sid}")
        except Exception as e:
            print(f"   ⚠️  Could not delete session {sid}: {e}")
    print("✅ Session cleanup complete")

print("✅ Session tracking utilities ready")

✅ Session tracking utilities ready


In [23]:
# Create a specialized agent with custom instructions
cloud_expert_agent = Agent(
    client=client,
    model=model_id,
    instructions="""You are an Oracle Cloud Infrastructure (OCI) expert assistant.
You help users understand and work with OCI services including:
- Compute instances and shapes
- Networking (VCN, subnets, security lists)
- Storage (Block, Object, File)
- AI/ML services and GPU instances
- Identity and Access Management (IAM)

Always provide practical, actionable advice with examples when possible.
If you're unsure about something, say so clearly.""",
)

print("✅ Created OCI Expert Agent")

✅ Created OCI Expert Agent


In [24]:
# Create a session for our agent
session_id = cloud_expert_agent.create_session(session_name="oci_consultation")
track_session(session_id)
print(f"✅ Created session: {session_id}")

INFO:httpx:HTTP Request: POST http://localhost:8321/v1/conversations "HTTP/1.1 200 OK"


✅ Created session: conv_b4f4097126e0759eac387b6e898e487d3490f6654381933b


In [25]:
# Helper function for chatting with agents

def chat_with_agent(
    agent: Agent,
    session_id: str,
    message: str,
    stream: bool = True,
    verbose: bool = False
) -> str:
    """
    Send a message to an agent and get the full response.
    Properly handles both text responses and tool calls.

    Args:
        agent: The Agent instance to chat with
        session_id: The session ID for the conversation
        message: The user message to send
        stream: Whether to stream the response (default: True)
        verbose: Whether to print debug information (default: False)

    Returns:
        The agent's response text

    Raises:
        ValueError: If agent, session_id, or message is invalid
    """
    if not agent:
        raise ValueError("Agent cannot be None")
    if not session_id:
        raise ValueError("Session ID cannot be empty")
    if not message:
        raise ValueError("Message cannot be empty")

    print(f"\n👤 User: {message}\n")
    print("🤖 Agent: ", end='')

    response = agent.create_turn(
        session_id=session_id,
        messages=[{"role": "user", "content": message}],
        stream=stream,
    )

    streamed_text = ""
    tool_calls_made = []

    for chunk in response:
        event = chunk.event
        event_type = event.event_type

        if verbose:
            print(f"\n[DEBUG] Event type: {event_type}")
            print(f"[DEBUG] Event: {event}")

        if event_type == "step_started":
            if hasattr(event, 'step_type'):
                if event.step_type == "tool_execution":
                    print("\n    Executing tool...", end='')

        elif event_type == "step_progress":
            if hasattr(event, 'delta'):
                if hasattr(event.delta, 'text') and event.delta.text:
                    text = event.delta.text
                    streamed_text += text
                    print(text, end='', flush=True)

        elif event_type == "step_completed":
            if hasattr(event, 'step_details'):
                step = event.step_details
                if hasattr(step, 'tool_calls') and step.tool_calls:
                    for tc in step.tool_calls:
                        tool_name = tc.tool_name if hasattr(tc, 'tool_name') else 'unknown'
                        tool_calls_made.append(tool_name)
                        print(f"\n   ✅ Tool '{tool_name}' completed")

        elif event_type == "turn_completed":
            if hasattr(event, 'turn') and hasattr(event.turn, 'output_message'):
                output = event.turn.output_message
                if hasattr(output, 'content') and output.content:
                    if isinstance(output.content, list):
                        for block in output.content:
                            if hasattr(block, 'text') and block.text:
                                if not streamed_text:
                                    print(block.text)
                                    streamed_text = block.text
                    elif hasattr(output.content, 'text'):
                        if not streamed_text:
                            print(output.content.text)
                            streamed_text = output.content.text
            break

    print("\n")

    if tool_calls_made:
        print(f"🔧 Tools used: {', '.join(tool_calls_made)}")

    return streamed_text

In [26]:
# Continue the conversation - the agent maintains context!
response = chat_with_agent(
    cloud_expert_agent,
    session_id,
    "How do I choose between them for training a large language model?"
)

INFO:httpx:HTTP Request: POST http://localhost:8321/v1/responses "HTTP/1.1 200 OK"



👤 User: How do I choose between them for training a large language model?

🤖 Agent: Training a Large Language Model (LLM) is a resource-intensive task that requires careful selection and configuration of OCI services. The right choice depends on your model's size, training data volume, budget, parallelism strategy, and desired level of infrastructure management.

Here's a breakdown of OCI services and how to choose between them for LLM training:

---

### **1. Compute (GPUs) - The Brain of Your LLM Training**

This is the most critical component. LLM training heavily relies on high-performance GPUs with ample VRAM and fast interconnects.

*   **Option A: Bare Metal GPU Instances (BM.GPU.A100-v2)**
    *   **What it is:** Dedicated physical servers with multiple NVIDIA A100 GPUs directly attached. The `BM.GPU.A100-v2` shape, for example, features **8x NVIDIA A100 80GB PCIe GPUs**. Crucially, these instances are provisioned within a **cluster network** fabric which enables **RDMA over C

---
## 2. Specialized Agent Personas

You can create multiple agents with different expertise for different tasks.

In [27]:
# Create a security-focused agent
security_agent = Agent(
    client=client,
    model=model_id,
    instructions="""You are an OCI Security Specialist. Your expertise includes:
- Identity and Access Management (IAM) policies
- Network security (Security Lists, NSGs, WAF)
- Encryption (Vault, Key Management)
- Security Zones and Cloud Guard
- Compliance and auditing

Always emphasize security best practices and zero-trust principles.
Warn users about potential security risks in their configurations.
Provide specific policy examples when relevant.""",
)

# Create a DevOps agent
devops_agent = Agent(
    client=client,
    model=model_id,
    instructions="""You are an OCI DevOps Engineer. Your expertise includes:
- CI/CD pipelines with OCI DevOps
- Container services (OKE, Container Instances)
- Infrastructure as Code (Terraform, Resource Manager)
- Monitoring and observability
- Automation and scripting with OCI CLI/SDK

Provide practical, automation-focused advice.
Include code snippets and CLI commands when helpful.
Emphasize repeatability and infrastructure as code.""",
)

print("✅ Created specialized agents:")
print("   • Security Agent")
print("   • DevOps Agent")

✅ Created specialized agents:
   • Security Agent
   • DevOps Agent


In [28]:
# Create sessions for each agent
security_session = security_agent.create_session(session_name="security_review")
track_session(security_session)
devops_session = devops_agent.create_session(session_name="devops_planning")
track_session(devops_session)

print("✅ Sessions created")

INFO:httpx:HTTP Request: POST http://localhost:8321/v1/conversations "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:8321/v1/conversations "HTTP/1.1 200 OK"


✅ Sessions created


In [29]:
# Ask the security agent
print("=" * 80)
print("SECURITY AGENT")
print("=" * 80)
response = chat_with_agent(
    security_agent,
    security_session,
    "What IAM policies do I need to set up for a team that manages AI workloads?"
)

INFO:httpx:HTTP Request: POST http://localhost:8321/v1/responses "HTTP/1.1 200 OK"


SECURITY AGENT

👤 User: What IAM policies do I need to set up for a team that manages AI workloads?

🤖 Agent: Managing AI workloads in OCI involves a diverse set of services, from core machine learning platforms to underlying compute, storage, and networking. Implementing robust IAM policies is crucial to maintain security, adhere to zero-trust principles, and ensure the team has precisely the access they need, and no more.

### Zero-Trust Foundation

Before defining specific policies, establish a strong foundation:

1.  **Dedicated Compartment**: Create a dedicated compartment (e.g., `ai-workloads`) for all AI-related resources. This simplifies policy management and isolation.
2.  **Dedicated IAM Group**: Create an IAM group for your AI team (e.g., `AI_Workloads_Team`). All policies will be applied to this group.
3.  **Principle of Least Privilege**: Start with minimal access and add permissions only as needed. Avoid `manage ALL resources` at the tenancy level.

---

### IAM Policies 

In [30]:
# Ask the DevOps agent the same general area but different focus
print("=" * 80)
print("DEVOPS AGENT")
print("=" * 80)
response = chat_with_agent(
    devops_agent,
    devops_session,
    "How do I set up a CI/CD pipeline for deploying ML models on OCI?"
)

INFO:httpx:HTTP Request: POST http://localhost:8321/v1/responses "HTTP/1.1 200 OK"


DEVOPS AGENT

👤 User: How do I set up a CI/CD pipeline for deploying ML models on OCI?

🤖 Agent: As an OCI DevOps Engineer, setting up a CI/CD pipeline for ML model deployment on OCI is a core task that leverages several services to ensure repeatability, reliability, and automation. The goal is to take trained ML models, package them, and deploy them to an inference endpoint with minimal manual intervention.

Here's a comprehensive guide, focusing on automation, IaC, and OCI services:

## CI/CD Pipeline for ML Model Deployment on OCI

### Core Principles

1.  **Infrastructure as Code (IaC):** Define your OCI resources (VCNs, IAM, OKE clusters, Container Registry, Data Science Projects, etc.) using Terraform. This ensures your infrastructure is version-controlled, repeatable, and consistent across environments.
2.  **Environment Parity:** Strive for identical (or near-identical) environments for development, staging, and production.
3.  **Automation First:** Every step that can be autom

---
## 3. Multi-Agent Collaboration Pattern

For complex tasks, you can coordinate multiple agents to work together.

In [31]:
def multi_agent_consultation(question: str) -> dict:
    """
    Get perspectives from multiple specialized agents on a question.
    """
    print(f"\n{'='*80}")
    print(f" MULTI-AGENT CONSULTATION")
    print(f"{'='*80}")
    print(f"\n Question: {question}\n")

    responses = {}

    # Get security perspective
    print("\n SECURITY PERSPECTIVE:")
    print("-" * 40)
    sec_session = security_agent.create_session(session_name="consultation_security")
    track_session(sec_session)
    response = security_agent.create_turn(
        session_id=sec_session,
        messages=[{"role": "user", "content": f"From a security perspective: {question}"}],
        stream=True,
    )
    sec_response = ""
    for chunk in response:
        if chunk.event.event_type == "turn_completed":
            sec_response = chunk.event.final_text
            break
        elif chunk.event.event_type == "step_progress":
            if hasattr(chunk.event.delta, 'text'):
                print(chunk.event.delta.text, end='', flush=True)
    responses['security'] = sec_response
    print("\n")

    # Get DevOps perspective
    print("\n DEVOPS PERSPECTIVE:")
    print("-" * 40)
    devops_sess = devops_agent.create_session(session_name="consultation_devops")
    track_session(devops_sess)
    response = devops_agent.create_turn(
        session_id=devops_sess,
        messages=[{"role": "user", "content": f"From a DevOps perspective: {question}"}],
        stream=True,
    )
    devops_response = ""
    for chunk in response:
        if chunk.event.event_type == "turn_completed":
            devops_response = chunk.event.final_text
            break
        elif chunk.event.event_type == "step_progress":
            if hasattr(chunk.event.delta, 'text'):
                print(chunk.event.delta.text, end='', flush=True)
    responses['devops'] = devops_response
    print("\n")

    return responses

print("✅ Multi-agent consultation function ready")

✅ Multi-agent consultation function ready


In [32]:
# Run a multi-agent consultation
results = multi_agent_consultation(
    "How should I architect a production ML inference service on OCI?"
)

INFO:httpx:HTTP Request: POST http://localhost:8321/v1/conversations "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:8321/v1/responses "HTTP/1.1 200 OK"



 MULTI-AGENT CONSULTATION

 Question: How should I architect a production ML inference service on OCI?


 SECURITY PERSPECTIVE:
----------------------------------------
Architecting a production ML inference service on OCI from a security perspective requires a multi-layered, defense-in-depth approach, heavily emphasizing **Zero-Trust principles**. The goal is to secure every component, from network ingress to data storage, and ensure continuous monitoring and auditing.

Here's a breakdown of how to
architect your service securely on OCI:

---

## Zero-Trust Foundation

Before diving into specifics, embed these Zero-Trust principles throughout your design:

1.  **Verify Explicitly:** Authenticate and authorize *every* request, whether from inside or outside your network perimeter.
2.  **Least Privilege Access:** Grant only the minimum necessary permissions to users, services, and applications at all times.
3.  **Assume Breach:** Design your security controls with the expectation that 

INFO:httpx:HTTP Request: POST http://localhost:8321/v1/conversations "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:8321/v1/responses "HTTP/1.1 200 OK"





 DEVOPS PERSPECTIVE:
----------------------------------------
Architecting a production ML inference service on OCI from a DevOps perspective centers around automation, reliability, scalability, and maintainability. The goal is to treat your ML model and its serving infrastructure as a single, version-controlled entity, deployed and managed through codified processes.

Here’s a comprehensive approach, prioritizing OCI-native services and best practices:

## Core Principles from a DevOps Perspective

1.  **Infrastructure as Code (IaC):** Everything—VCN, OKE clusters, API Gateways, load balancers, even monitoring configurations—should be defined in Terraform. This ensures repeatability, version control, and immutability.
2.  **Containerization:** Package your ML model and its serving application (e.g., Flask/FastAPI) into immutable Docker images.
3.  **CI/CD Automation:** Use OCI DevOps services to automate building, testing, deploying, and managing your inference service.
4.  **Obser

---
## 4. Agentic Workflow: Infrastructure Review

Let's add an architect agent that reviews the responses of the security and devops agents

In [33]:
# Create an architect agent that synthesizes recommendations
architect_agent = Agent(
    client=client,
    model=model_id,
    instructions="""You are a Senior OCI Solutions Architect. Your role is to:
- Review infrastructure requirements and constraints
- Synthesize input from security and DevOps perspectives
- Provide comprehensive, balanced architectural recommendations
- Consider cost, performance, security, and operational aspects
- Create actionable implementation plans

You provide executive-level summaries followed by detailed technical guidance.""",
)

print("✅ Created Architect Agent")

✅ Created Architect Agent


In [34]:
def infrastructure_review_workflow(requirements: str) -> dict:
    """
    A multi-stage workflow for infrastructure review.

    Stage 1: Security Analysis
    Stage 2: DevOps Analysis
    Stage 3: Architecture Synthesis
    """
    print("\n" + "="*80)
    print("  INFRASTRUCTURE REVIEW WORKFLOW")
    print("="*80)
    print(f"\n Requirements:\n{requirements}\n")

    # Stage 1: Security Analysis
    print("\n" + "="*80)
    print(" STAGE 1: Security Analysis")
    print("="*80)
    sec_session = security_agent.create_session(session_name="review_security")
    track_session(sec_session)
    sec_response = security_agent.create_turn(
        session_id=sec_session,
        messages=[{"role": "user", "content": f"Review these requirements for security considerations:\n\n{requirements}\n\nProvide key security requirements and concerns."}],
        stream=True,
    )
    security_analysis = ""
    for chunk in sec_response:
        if chunk.event.event_type == "turn_completed":
            security_analysis = chunk.event.final_text
            break
        elif chunk.event.event_type == "step_progress":
            if hasattr(chunk.event.delta, 'text'):
                print(chunk.event.delta.text, end='', flush=True)
    print("\n")

    # Stage 2: DevOps Analysis
    print("\n" + "="*80)
    print(" STAGE 2: DevOps Analysis")
    print("="*80)
    devops_sess = devops_agent.create_session(session_name="review_devops")
    track_session(devops_sess)
    devops_response = devops_agent.create_turn(
        session_id=devops_sess,
        messages=[{"role": "user", "content": f"Review these requirements for DevOps implementation:\n\n{requirements}\n\nProvide key operational requirements and automation opportunities."}],
        stream=True,
    )
    devops_analysis = ""
    for chunk in devops_response:
        if chunk.event.event_type == "turn_completed":
            devops_analysis = chunk.event.final_text
            break
        elif chunk.event.event_type == "step_progress":
            if hasattr(chunk.event.delta, 'text'):
                print(chunk.event.delta.text, end='', flush=True)
    print("\n")

    # Stage 3: Architecture Synthesis
    print("\n" + "="*80)
    print(" STAGE 3: Architecture Synthesis")
    print("="*80)
    arch_session = architect_agent.create_session(session_name="review_synthesis")
    track_session(arch_session)
    synthesis_prompt = f"""Based on the following inputs, provide a comprehensive infrastructure recommendation:

## Original Requirements:
{requirements}

## Security Analysis:
{security_analysis}

## DevOps Analysis:
{devops_analysis}

Provide a synthesized recommendation that addresses all concerns with a clear implementation plan."""

    arch_response = architect_agent.create_turn(
        session_id=arch_session,
        messages=[{"role": "user", "content": synthesis_prompt}],
        stream=True,
    )
    final_recommendation = ""
    for chunk in arch_response:
        if chunk.event.event_type == "turn_completed":
            final_recommendation = chunk.event.final_text
            break
        elif chunk.event.event_type == "step_progress":
            if hasattr(chunk.event.delta, 'text'):
                print(chunk.event.delta.text, end='', flush=True)
    print("\n")

    print("\n" + "="*80)
    print("✅ REVIEW COMPLETE")
    print("="*80)

    return {
        "security_analysis": security_analysis,
        "devops_analysis": devops_analysis,
        "final_recommendation": final_recommendation
    }

print("✅ Infrastructure review workflow ready")

✅ Infrastructure review workflow ready


In [35]:
# Run the infrastructure review workflow
requirements = """
We need to deploy a real-time ML inference service with the following requirements:

1. Handle 1000 requests per second at peak
2. Maximum latency of 100ms for 99th percentile
3. Use GPU-accelerated inference
4. Support model updates without downtime
5. Store inference logs for 90 days
6. Operate in us-phoenix-1 region
7. Budget: ~$10,000/month
"""

review_results = infrastructure_review_workflow(requirements)

INFO:httpx:HTTP Request: POST http://localhost:8321/v1/conversations "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:8321/v1/responses "HTTP/1.1 200 OK"



  INFRASTRUCTURE REVIEW WORKFLOW

 Requirements:

We need to deploy a real-time ML inference service with the following requirements:

1. Handle 1000 requests per second at peak
2. Maximum latency of 100ms for 99th percentile
3. Use GPU-accelerated inference
4. Support model updates without downtime
5. Store inference logs for 90 days
6. Operate in us-phoenix-1 region
7. Budget: ~$10,000/month



 STAGE 1: Security Analysis
Deploying a real-time ML inference service on OCI with your specified requirements (high throughput, low latency, GPU acceleration, zero-downtime updates, log retention, budget) presents several critical security considerations. As an OCI Security Specialist, my focus will be on embedding security from the ground up, adhering strictly to Zero Trust principles, and emphasizing robust controls.

Here are the key security requirements and concerns, broken down by domain:

---

### **Zero Trust Foundation**

Before diving into specifics, it's crucial to establish a Zer

INFO:httpx:HTTP Request: POST http://localhost:8321/v1/conversations "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:8321/v1/responses "HTTP/1.1 200 OK"





 STAGE 2: DevOps Analysis
As an OCI DevOps Engineer, I've reviewed your requirements for the real-time ML inference service. This is an exciting challenge that leverages many of OCI's strengths in compute, networking, and CI/CD. The key here is achieving high performance, reliability, and automated operations within budget.

Here's a breakdown of the key operational requirements, architectural considerations, and automation opportunities, emphasizing repeatability and Infrastructure as Code (IaC).

---

## 1. Deconstructing Requirements and OCI Service Mapping

| Requirement                          | OCI Service/Feature                                   | Operational Implication                              |
| :----------------------------------- | :---------------------------------------------------- | :--------------------------------------------------- |
| 1. Handle 1000 RPS at peak           | OCI Container

INFO:httpx:HTTP Request: POST http://localhost:8321/v1/conversations "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:8321/v1/responses "HTTP/1.1 200 OK"





 STAGE 3: Architecture Synthesis
## Real-time ML Inference Service on OCI: Comprehensive Architectural Recommendation

### Executive Summary

This document outlines a robust, secure, and operationally efficient infrastructure architecture for deploying your real-time ML inference service on Oracle Cloud Infrastructure (OCI). Our recommendation leverages OCI's native services to meet high-performance requirements (1000 RPS, P99 < 100ms latency), GPU acceleration, zero-downtime model updates, and comprehensive logging, all while adhering to strong security principles and DevOps best practices.

The core of the architecture will revolve around **OCI Container Engine for Kubernetes (OKE)** running GPU-enabled worker nodes for flexible, scalable inference deployment. It will be fronted by an **OCI Load Balancer** and protected by an **OCI Web Application Firewall (WAF)** for ingress traffic. **OCI Object Storage** will securely house ML models and inference logs, integrated with **OCI Lo

---
## 5. Cleanup

Clean up all sessions created during this notebook run.

In [36]:
# Clean up all tracked sessions
print(f"📊 Total sessions created: {len(all_sessions)}")
cleanup_sessions(all_sessions)

INFO:httpx:HTTP Request: DELETE http://localhost:8321/v1/conversations/conv_b4f4097126e0759eac387b6e898e487d3490f6654381933b "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: DELETE http://localhost:8321/v1/conversations/conv_0ebd02430a7d71a10ef063c25ad60f68e2ff0a2510d4b4e3 "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: DELETE http://localhost:8321/v1/conversations/conv_ffd7ea8c7cce7d2fbe76b96bda4d38c45809e189178d2e23 "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: DELETE http://localhost:8321/v1/conversations/conv_a54387c8c21f9f1f05d030695c5256f359cd8847a46f6abb "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: DELETE http://localhost:8321/v1/conversations/conv_c85584e0c7fc03a4be822479d0d96805bd46e311ef4cf9af "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: DELETE http://localhost:8321/v1/conversations/conv_db43f1c47d76959cb34de94922a08ab80167240a01977be5 "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: DELETE http://localhost:8321/v1/conversations/conv_1c3d69c80ea5a4a4ce613b09708a9c7a582f595e2f00702b "HTTP/1.1 200 OK"

📊 Total sessions created: 8
🧹 Cleaning up 8 session(s)...
   ✅ Deleted session: conv_b4f4097126e0759eac387b6e898e487d3490f6654381933b
   ✅ Deleted session: conv_0ebd02430a7d71a10ef063c25ad60f68e2ff0a2510d4b4e3
   ✅ Deleted session: conv_ffd7ea8c7cce7d2fbe76b96bda4d38c45809e189178d2e23
   ✅ Deleted session: conv_a54387c8c21f9f1f05d030695c5256f359cd8847a46f6abb
   ✅ Deleted session: conv_c85584e0c7fc03a4be822479d0d96805bd46e311ef4cf9af
   ✅ Deleted session: conv_db43f1c47d76959cb34de94922a08ab80167240a01977be5
   ✅ Deleted session: conv_1c3d69c80ea5a4a4ce613b09708a9c7a582f595e2f00702b
   ✅ Deleted session: conv_a23953cebc92f017349cf569b9537e1f6ca84357b8c465d6
✅ Session cleanup complete
